In [ ]:
#Algorítimo de Árvores de decisão

In [ ]:
# Importação das bibliotecas
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import *
import matplotlib.pyplot as plt

import mlflow
import mlflow.sklearn

In [ ]:
# Carregamento da base de dados e definição dos previsores (variáveis independentes - X) e classe (variável dependente - y)
zabbix = pd.read_csv('teste.csv',delimiter=";")
zabbix.shape

In [ ]:
# Mostra o tipo das variáveis
zabbix.dtypes

In [ ]:
# Retirando as informações da tabela
zabbix.drop('eventid', axis=1, inplace=True)
zabbix.drop('FROM_UNIXTIME(p.clock)', axis=1, inplace=True)
zabbix.drop('FROM_UNIXTIME(p.r_clock)', axis=1, inplace=True)
#zabbix.shape

In [ ]:
zabbix.head()

In [ ]:
# Conversão dos atributos categóricos para atributos numéricos
for col in zabbix.columns:
    if zabbix[col].dtype == 'object':
        zabbix[col] = zabbix[col].astype('category').cat.codes

In [ ]:
zabbix.head()

In [ ]:
previsores = zabbix.iloc[:,0:4].values
classe = zabbix.iloc[:,4].values

In [ ]:
previsores

In [ ]:
# Divisão da base de dados entre treinamento e teste. Usamos 30% para testar e 70% para treinar. Random_state = 0 para sempre obter a mesma divisão da base quando o código for executado
X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(previsores,classe,
                                        test_size=0.5,random_state=0)

In [ ]:
def treina_rf(n_estimators):
    mlflow.set_experiment("zbxexperimento")
    with mlflow.start_run():
        arvore = RandomForestClassifier(n_estimators=n_estimators)
        arvore.fit(X_treinamento, y_treinamento)
        previsoes = arvore.predict(X_teste) 

        #log de hiper parametros
        mlflow.log_param("n_estimators", n_estimators)    
    
        #métrica
        acuracia = accuracy_score(y_teste, previsoes)
    
        #registrar métrica
        mlflow.log_metric("acuracia",acuracia)
    
        #gráfico
        confusion = plot_confusion_matrix(arvore, X_teste, y_teste)
        plt.savefig("confusion.png")
    
        #logar gráfico
        mlflow.log_artifact("confusion.png")
    
        #modelo
        mlflow.sklearn.log_model(arvore,"ModeloZBX")
    
    
        #informações da execução
        print("Modelo: ", mlflow.active_run().info.run_uuid)    
    mlflow.end_run()

In [ ]:
arvores = [20,80,600,4000,8000,50000,100000]
for n in arvores:
    treina_rf(n)